<a href="https://colab.research.google.com/github/linlcc/NLP/blob/master/4_CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.


In [0]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [0]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2020-04-15 20:16:50--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.1’

shakespeare_plays.t 100%[===================>]   4.53M  19.0MB/s    in 0.2s    

2020-04-15 20:16:50 (19.0 MB/s) - ‘shakespeare_plays.txt.1’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [0]:
model = Word2Vec(lines)

Even with such a small training set size, I can perform some standard analogy tasks.

In [0]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.8442368507385254),
 ('prince', 0.7322368621826172),
 ('warwick', 0.7289110422134399),
 ('duke', 0.7008894681930542),
 ('margaret', 0.6998493671417236),
 ('widow', 0.6977822780609131),
 ('clarence', 0.6935532689094543),
 ('gloucester', 0.6768079996109009),
 ('york', 0.6758090257644653),
 ('elizabeth', 0.664851188659668)]

For the rest of this, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [0]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('desdemona', 0.9487350583076477),
 ('iago', 0.9231839776039124),
 ('ham', 0.9201522469520569),
 ('cleopatra', 0.9169795513153076),
 ('proteus', 0.9152540564537048),
 ('cressida', 0.9049752950668335),
 ('anne', 0.9027912616729736),
 ('valentine', 0.9011837244033813),
 ('pisanio', 0.9009215831756592),
 ('troilus', 0.8984599113464355)]

This search uses cosine similarity. In the default API, we could see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [0]:
model.wv.similarity('othello', 'desdemona')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9487352

Implement our own cosine similarity function so that I can reuse it outside of the context of the gensim model object.

In [0]:
## Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return (np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))

## Should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.94873506

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [0]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2020-04-15 20:17:13--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.1’

30k.en.vec.1        100%[===================>]  64.54M  74.0MB/s    in 0.9s    

2020-04-15 20:17:14 (74.0 MB/s) - ‘30k.en.vec.1’ saved [67681172/67681172]

--2020-04-15 20:17:15--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.1’

30k.fr.vec.1        100%[===================>]  64.66M  65.2MB/s    in 1.0s    

2020-04-15 20:17:16 (65.2 MB/s) - ‘30k.fr.vec.1’ saved [67802327/67802327]

--2020-04-15 20:17:16--  http://www.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [0]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# load vectors for one more language, such as zhvec
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
zhvec = ftvectors('30k.zh.vec')

Write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this, I use a linear scan through the dictionary using your `cosim` function from above.

In [0]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  re_dict={}
  for item in vecDict:
    a=vecDict[item]
    re=cosim(vec,a)
    re_dict[item]=re
  dist_sort=sorted(re_dict.items(), key=lambda dist: dist[1],reverse = True)
  mostSimilar = dist_sort[0][0]
  similarity = dist_sort[0][1]

  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]
# [mostSimilar(envec[e], zhvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If we looked further down the list, we would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [0]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2020-04-15 20:17:47--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.1’

links.tab.1         100%[===================>]   1.34M  7.71MB/s    in 0.2s    

2020-04-15 20:17:47 (7.71 MB/s) - ‘links.tab.1’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [0]:
links[302]

['academy', 'fr', 'académie']

Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [0]:

def norm_vec(vecDict):
  mod_vec={}
  for item in vecDict:
    mod_vec[item]=np.linalg.norm(vecDict[item])
  return mod_vec

def cosim2(v1, v2, v3, v4):
  return (np.dot(v1,v2)/(v3*v4))

def mostSimilar2(vec,normvec, vecDict,normvecdict):
  re_dict={}
  for item in vecDict:
    a=vecDict[item]
    b=normvecdict[item]
    re=cosim2(vec,a,normvec,b)
    re_dict[item]=re
  dist_sort=sorted(re_dict.items(), key=lambda dist: dist[1],reverse = True)
  mostSimilar = dist_sort[0][0]
  similarity = dist_sort[0][1]
  return (mostSimilar, similarity)

# ## some example searches
# norm_envec=norm_vec(envec)
# norm_frvec=norm_vec(frvec)
# [mostSimilar2(envec[e],norm_envec[e],frvec,norm_frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

def evaluate(vecdict1,vecdict2,targetvec):
  norm_vecdict1=norm_vec(vecdict1)
  norm_vecdict2=norm_vec(vecdict2)
  i=0
  j=0
  k=0
  for item in links:
    if item[1]==targetvec:
      i=i+1
      if item[0]==item[2]:
        k=k+1
      a=mostSimilar2(vecdict1[item[0]],norm_vecdict1[item[0]],vecdict2,norm_vecdict2)
      if a[0]==item[2]:
        j=j+1
  return {'accuracy': j/i,
          'baselineAccuracy':k/i}

In [0]:
evaluate(envec,frvec,'fr')

{'accuracy': 0.5359205593271862, 'baselineAccuracy': 0.6742324450298915}

Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [0]:
## Compute English-X Wikipedia retrieval accuracy.
evaluate(envec,zhvec,'zh')

{'accuracy': 0.13639637044505257, 'baselineAccuracy': 0.06740602045225406}